# Libraries

In [2]:
import os
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from pprint import pprint

from src.utils import (
    # Old utils
    print_h, eval_window, eval_person_majority_voting,

    # New utils
    set_seed, get_device, init_metrics, update_metrics, save_metrics_to_json,
    plot_k_fold_roc_curves_multiclass_v2, plot_k_fold_cm,
)
from src.models import RNNInceptionTime, HardMoE

# Config

In [3]:
# Project config
seed = 69
set_seed(seed)
device = get_device()
print("Device:", device)

# Model config
fold_i_dir_map = {
    'Ga': 'datasets/preprocessed/Ga_k10_w500_s500_v20250430232239/fold_09',
    'Ju': 'datasets/preprocessed/Ju_k10_w500_s500_v20250430232219/fold_09',
    'Si': 'datasets/preprocessed/Si_k10_w500_s250_v20250430231926/fold_09',
}
expert_model_dir_map = {
    'Ga': 'checkpoints/RNNInceptionTime_bidirectional_Ga_k10_w500_s500_fold_09_e1_v20250514220024/fold_09.pth',
    'Ju': 'checkpoints/RNNInceptionTime_bidirectional_Ju_k10_w500_s500_fold_09_e1_v20250514220117/fold_09.pth',
    'Si': 'checkpoints/RNNInceptionTime_bidirectional_Si_k10_w500_s250_fold_09_e1_v20250514220147/fold_09.pth'
}
gate_model_dir = 'checkpoints/MLPGate_l2_Ga_k10_w500_s500_Ju_k10_w500_s500_Si_k10_w500_s250_fold_09_e1_v20250514224321/fold_09.pth'

# Set up model path mappings and get number of folds (K-fold)
expert_model_path_map = {study: sorted([model_dir_study+'/'+f for f in os.listdir(model_dir_study) if f.endswith('.pth')])
                         for study, model_dir_study in expert_model_dir_map.items()}
gate_model_path_map = sorted([gate_model_dir+'/'+f for f in os.listdir(gate_model_dir) if f.endswith('.pth')])
assert len(set([len(expert_model_path_study) for expert_model_path_study in expert_model_path_map.values()])) == 1, \
    f"Inconsistent number of folds across dataset studies: {[len(v) for v in expert_model_path_map.values()]}"
k_fold = len(list(expert_model_path_map.values())[0])
print("K-fold:", k_fold)

# Evaluation config
k_fold = 10
batch_size = 8
n_feat = 16
n_class = 4
window_size = 500
max_vgrf_data_len = 25_000
general_metrics_dir = 'evaluations/RNNInceptionTimeMoE_' + gate_model_dir.rsplit('RNNInceptionTime_')[-1].split('_v')[0] + '/_general_metrics'
cm_dir = 'evaluations/RNNInceptionTimeMoE_' + gate_model_dir.rsplit('RNNInceptionTime_')[-1].split('_v')[0] + '/cm'
roc_curves_dir = 'evaluations/RNNInceptionTimeMoE_' + gate_model_dir.rsplit('RNNInceptionTime_')[-1].split('_v')[0] + '/roc_curves'

print("Evaluation general metrics save directory:", general_metrics_dir)
print("Evaluation confusion matrix save directory:", cm_dir)
print("Evaluation ROC curves save directory:", roc_curves_dir)

Random seed: 69
Device: mps


FileNotFoundError: [Errno 2] No such file or directory: '../Parkinsons_VGRF_Spatiotemporal_Diagnosis_v0/outputs/train/RNNInceptionTime_Ga_baseline_20240911161249'